In [1]:
# import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import math

/Users/katherinemcphie/anaconda3/envs/testEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/katherinemcphie/anaconda3/envs/testEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/katherinemcphie/anaconda3/envs/testEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/kather

In [79]:
# loading the data
mushroom_data = pd.read_csv('./mushrooms.csv')

# the goal is to use machine learning to predict whether or not different mushrooms are poisonous
# data was taken from kaggle.com and classifies different mushrooms based on various characteristics

In [81]:
# understanding the data
mushroom_data.head()
mushroom_data.info()

mushroom_Y = pd.get_dummies(mushroom_data.iloc[:,0], drop_first = False)
mushroom_X = pd.DataFrame()
for i in mushroom_data.iloc[:,1:].columns:
    j = pd.get_dummies(mushroom_data[i], prefix = i, drop_first = False)
    mushroom_X = pd.concat([mushroom_X, j], axis = 1)
    
mushroom_X.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [82]:
# splitting into test and train
train_percent = 0.8

mushroom_cutoff = math.floor(len(mushroom_X) * train_percent)

trainX = mushroom_X[:mushroom_cutoff]
trainY = mushroom_Y[:mushroom_cutoff]
testX = mushroom_X[mushroom_cutoff:]
testY = mushroom_Y[mushroom_cutoff:]

trainX.shape # 117
testX.shape 

(1625, 117)

In [83]:
# building out the model
model = keras.models.Sequential()

# adding the first hidden layer
model.add(layers.Dense(12, input_shape=(117,))) # 12 nodes
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.2))  # dropout layer is to avoid overfitting

# adding the second hidden layer 
model.add(layers.Dense(8)) # input shape already inferred
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.2))

# adding the output layer
model.add(layers.Dense(2)) # trying to predict 2 things (poisonous or not poisonous)
model.add(layers.Activation('softmax')) # picks best guess as answer

In [84]:
# compiling the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adamax')

In [86]:
# training the model
history = model.fit(trainX, trainY,
          batch_size=128, epochs=10,
          verbose=2,
          validation_data=(testX, testY))

Train on 6499 samples, validate on 1625 samples
Epoch 1/10
6499/6499 - 0s - loss: 0.0228 - acc: 0.9952 - val_loss: 0.0766 - val_acc: 0.9674
Epoch 2/10
6499/6499 - 0s - loss: 0.0228 - acc: 0.9934 - val_loss: 0.0775 - val_acc: 0.9655
Epoch 3/10
6499/6499 - 0s - loss: 0.0213 - acc: 0.9951 - val_loss: 0.0710 - val_acc: 0.9686
Epoch 4/10
6499/6499 - 0s - loss: 0.0195 - acc: 0.9960 - val_loss: 0.0704 - val_acc: 0.9692
Epoch 5/10
6499/6499 - 0s - loss: 0.0180 - acc: 0.9946 - val_loss: 0.0691 - val_acc: 0.9686
Epoch 6/10
6499/6499 - 0s - loss: 0.0160 - acc: 0.9960 - val_loss: 0.0665 - val_acc: 0.9692
Epoch 7/10
6499/6499 - 0s - loss: 0.0160 - acc: 0.9960 - val_loss: 0.0651 - val_acc: 0.9705
Epoch 8/10
6499/6499 - 0s - loss: 0.0162 - acc: 0.9954 - val_loss: 0.0638 - val_acc: 0.9705
Epoch 9/10
6499/6499 - 0s - loss: 0.0133 - acc: 0.9974 - val_loss: 0.0648 - val_acc: 0.9705
Epoch 10/10
6499/6499 - 0s - loss: 0.0122 - acc: 0.9968 - val_loss: 0.0646 - val_acc: 0.9698
